## 4. K-means

### 4.1. 개요

  - K-평균 알고리즘(K-means algorithm)은 주어진 데이터를 k개의 클러스터로 묶는 방법으로 각 클러스터와의 거리 차이 분산을 최소화
  - 본 알고리즘은 비지도학습(Unsupervised Learning)의 일종으로, 레이블이 없는 입력 데이터에 레이블을 달아주는 역할을 수행
  - K-평균 알고리즘은 공간데이터 분석(물류센터 위치선정), 웹문서 분류 등 다양한 분야 에서 이용
  - (위키피디아)

  <img src = "images/image15.png">

### 4.2. K-means 알고리즘

  - Assignment step 과 update step 으로 구성
  - Assignment step 에서는 점(point)으로부터 각 클러스터의 중심점까지의 직선 거리를 계산하고, 그 점에서 가장 가까운 클러스터를
찾아 점을 배당
  -  Update step 에서는 각 클러스터에 있는 점들의 무게중심 값으로 해당 클러스터의 중심위치를 재설정. 
  -  클러스터의 중심위치가 변하지 않을 때까지 반복
  
  <img src = "images/image16.png">

1. Scipy 함수를 이용하여 데이터 생성

In [5]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs

In [ ]:
X,y = make_blobs(n_samples = 500,n_features = 2,centers = 2,random_state = 23)
fig = plt.figure(0)
plt.grid(True)
plt.scatter(X[:,0], X[:,1])
plt.show()

In [ ]:
X,y = make_blobs(n_samples = 500,n_features = 2,centers = 10,random_state = 23)

np.save("examples/X", X)
np.save("examples/y", y)

fig = plt.figure(0)
plt.grid(True)
plt.scatter(X[:,0], X[:,1])
plt.show()


2. 순차코드
  - 7개로 생성된 클러스터를 분류
  - 초기 무게중심 값은 (-10, 10) 사에에서,랜덤하게 생성

In [ ]:
k = 7
 
clusters = {}
np.random.seed(23)
 
##### 딕셔너리 자료형을 이용하여 k개의 클러스터의 중심 좌표와 point 좌표 저장할 수 있는 객체 생성
for idx in range(k):
    center = 10*(2*np.random.random((X.shape[1],))-1)
    points = []
    cluster = {
        'center' : center,
        'points' : []
    }
     
    clusters[idx] = cluster
     
clusters

In [ ]:
def distance(p1,p2):
    return np.sqrt(np.sum((p1-p2)**2))

#Implementing E step 
def assign_clusters(X, clusters):
    # 각 데이터 포인트에 대해 반복
    for idx in range(X.shape[0]):
        dist = []
         
        curr_x = X[idx]
         
        # k번째 클러스터 센터와 각 데이터 포인트의 거리 계산
        for i in range(k):
            dis = distance(curr_x,clusters[i]['center'])
            dist.append(dis)

        # 계산된 클러스터와의 거리중에 최소값으로 클러스터를 배정
        curr_cluster = np.argmin(dist)

        # 현재 데이터 포인트의 좌표를 속한 클러스터에 추가
        clusters[curr_cluster]['points'].append(curr_x)
    return clusters
         
#Implementing the M-Step
def update_clusters(X, clusters):
    # 클러스터에 대해 반복
    for i in range(k):

        # 현재 클러스터에 포함된 데이터 포인트의 좌표를 불러옴
        points = np.array(clusters[i]['points'])

        # 데이터 포인트들의 평균을 취해 현재 클러스터 중심을 업데이트
        if points.shape[0] > 0:
            new_center = points.mean(axis =0)
            clusters[i]['center'] = new_center
             
            clusters[i]['points'] = []
    return clusters

# 데이터 포인트 X에 대해 가장 거리가 가까운 클러스터를 예측
def pred_cluster(X, clusters):
    pred = []
    for i in range(X.shape[0]):
        dist = []
        for j in range(k):
            dist.append(distance(X[i],clusters[j]['center']))
        pred.append(np.argmin(dist))
    return pred

# Repeat E-M steps
for i in range (10) :
    clusters = assign_clusters(X,clusters)
    clusters = update_clusters(X,clusters)
    print(clusters)

# Data point clustering
pred = pred_cluster(X,clusters)

plt.scatter(X[:,0],X[:,1],c = pred)
for i in clusters:
    center = clusters[i]['center']
    plt.scatter(center[0],center[1],marker = '^',c = 'red')
plt.show()

3. 병렬코드
  - 의존성 분석
    - 각 점의 거리계산은 독립적임
    - 거리계산으로부터 무게중심 계산시 reduction 연산이 필요
    - 출력을 위해 각 프로세스의 결과를 gather

      <img src = "images/image26.png">

  - 하나의 클러스터에서 중심을 계산하는 방법
    <img src = "images/image26_1.png">

In [ ]:
%%writefile examples/KM.py
import numpy as np
import matplotlib.pyplot as plt
from mpi4py import MPI
from sklearn.datasets import make_blobs
from tools import para_range

def distance(p1,p2):
    return np.sqrt(np.sum((p1-p2)**2))

#Implementing E step 
def assign_clusters(X, clusters):
    # 각 데이터 포인트에 대해 반복
    for idx in range(X.shape[0]):
        dist = []
         
        curr_x = X[idx]
         
        # k번째 클러스터 센터와 각 데이터 포인트의 거리 계산
        for i in range(k):
            dis = distance(curr_x,clusters[i]['center'])
            dist.append(dis)

        # 계산된 클러스터와의 거리중에 최소값으로 클러스터를 배정
        curr_cluster = np.argmin(dist)

        # 현재 데이터 포인트의 좌표를 속한 클러스터에 추가
        clusters[curr_cluster]['points'].append(curr_x)
    return clusters
         
#Implementing the M-Step
def update_clusters(X, clusters):
    # 클러스터에 대해 반복
    for i in range(k):

        # 현재 클러스터에 포함된 데이터 포인트의 좌표를 불러옴
        points = np.array(clusters[i]['points'])

        # 데이터 포인트 좌표의 합을 먼저 구함
        # Local sum
        coord_sum_local = points.sum(axis = 0)
        # 평균 계산을 위해 데이터 포인트 좌표의 합과 데이터 포인트 개수의 global sum을 구함
        n_local_points = comm.allreduce( #FIX ME
        coord_sum_global = comm.allreduce( #FIX ME

        # 데이터 포인트들의 평균을 취해 현재 클러스터 중심을 업데이트. 모든 랭크들이 같은 값을 가짐
        if n_local_points > 0:
            new_center = #FIX ME
            clusters[i]['center'] = new_center             
            clusters[i]['points'] = []

    return clusters

# 데이터 포인트 X에 대해 가장 거리가 가까운 클러스터를 예측
def pred_cluster(X, clusters):
    pred = []
    for i in range(X.shape[0]):
        dist = []
        for j in range(k):
            dist.append(distance(X[i],clusters[j]['center']))
        pred.append(np.argmin(dist))
    return pred

comm = MPI.COMM_WORLD

rank = comm.Get_rank()
size = comm.Get_size()

n = 500
k = 7

##### 데이터 포인트 불러오기
if rank == 0 :

    X = np.load("examples/X.npy")
    y = np.load("examples/y.npy")
    
    fig = plt.figure(0)
    plt.grid(True)
    plt.scatter(X[:,0], X[:,1])
    plt.show()

else : 
    X = np.empty((0, 0), dtype = np.float64)

clusters = {}
np.random.seed(23)

##### 임의로 클러스터 중심을 생성. 모든 랭크들은 동일한 값을 가지고 있음
for idx in range(k):
    center = 10*(2*np.random.random((2,))-1)
    points = []
    cluster = {
        'center' : center,
        'points' : []
    }
    
    clusters[idx] = cluster

##### 프로세스별 범위 할당
ista, iend = para_range(n, size, rank)
chunk = iend - ista + 1

#### 각 랭크가 가질 데이터 포인트를 선언
X_chunk = np.empty([chunk,2], np.float64)

##### 데이터를 Scatterv로 분할하기 위해 chunk를 이용한 리스트 생성. 이 때 x,y좌표를 위해 2를 곱합
chunk_cnts = comm.allgather(chunk*2)

##### 데이터를 chunk로 분할. cluster 정보는 공유 (데이터 양이 크지 않음)
comm.Scatterv( #FIX ME
comm.bcast( #FIX ME

for i in range (10) :
    clusters = assign_clusters( #FIX ME
    clusters = update_clusters( #FIX ME

    if rank == 0 :
        print(clusters)

# Data point clustering
pred = pred_cluster(X_chunk,clusters)

##### prediction gather
pred_all = comm.gather( #FIX ME

if rank == 0 :
    plt.figure(1)
    plt.scatter(X[:,0],X[:,1], c = pred_all)
    for i in clusters:
        center = clusters[i]['center']
        plt.scatter(center[0],center[1],marker = '^',c = 'red')
    plt.savefig("examples/KM_paralle.png")

In [ ]:
! mpiexec -np 4 python examples/KM.py